# 1. Load the Preprocessed Dataset

In [15]:
# Launch magic commands to automatically reload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Import the pandas, numpy packages and dump from joblib
import pandas as pd
import numpy as np
from joblib import dump

In [17]:
# Load the saved sets from data/processed
X_train = pd.read_csv('../data/processed/X_train.csv')
X_val   = pd.read_csv('../data/processed/X_val.csv'  )
X_test  = pd.read_csv('../data/processed/X_test.csv' )
y_train = pd.read_csv('../data/processed/y_train.csv')
y_val   = pd.read_csv('../data/processed/y_val.csv'  )

# 3. Train Logistic Regression model

In [18]:
# Import the logistic regression module from sklearn
from sklearn.linear_model import LogisticRegression

In [19]:
# Instantiate the LogisticRegression class into a variable called reg
reg = LogisticRegression()

In [20]:
# Fit the model with the prepared data
reg.fit(X_train, y_train)

C:\Users\this\AppData\Local\pypoetry\Cache\virtualenvs\adv-mla-at1-Dk-CNlDi-py3.9\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\this\AppData\Local\pypoetry\Cache\virtualenvs\adv-mla-at1-Dk-CNlDi-py3.9\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
# Save the fitted model into the folder models as a file called log_reg
dump(reg,  '../models/log_reg.joblib')

['../models/log_reg.joblib']

# 4. Model Evaluation

In [22]:
# Prediction probability on the training, validation and testing datasets
preds_proba_train = reg.predict_proba(X_train) 
preds_proba_val = reg.predict_proba(X_val) 
preds_proba_test = reg.predict_proba(X_test) 

In [23]:
# Keep the prediction probabilities for the drafted variable
preds_proba_train = preds_proba_train[:, 1]
preds_proba_val = preds_proba_val[:, 1]
preds_proba_test = preds_proba_test[:, 1]

In [24]:
# Import the roc_auc_score from scikit-learn to evaluate the model performance
from sklearn.metrics import roc_curve, roc_auc_score

In [25]:
# Calculate the AUROC score of the validation dataset
reg_auc_val = roc_auc_score(y_val, preds_proba_val)
reg_auc_val

0.9874001668651553

In [26]:
# Calculate the AUROC score of the training dataset
reg_auc_train = roc_auc_score(y_train, preds_proba_train)
reg_auc_train

0.9891360804890494

As the AUROC scores on training and validation datasets suggest, we can see that there is overfitting issue with the Logistic Regression model.

# 5. Submission File

In [27]:
# Open the testing dataset and save as df_test_cleaned
df_test_cleaned = pd.read_csv('../data/raw/advmla-2023-spring/test.csv')

In [28]:
# Split the column player_id from the df_test_cleaned and save as a variable called player_id
player_id = df_test_cleaned.pop('player_id')

In [30]:
# Create a dataframe containing 2 columns: player_id and preds_proba_test. Save as submission
submission = pd.DataFrame({'player_id': player_id, 'drafted': list(preds_proba_test)})
submission

,player_id,drafted
0,cf302b4d-84f7-4124-a25d-a75eed31978b,1.062179e-08
1,f91837cd-4f49-4b70-963d-aeb82c6ce3da,2.246725e-07
2,53ec2a29-1e7d-4c6d-86d7-d60d02af8916,6.998281e-05
3,32402798-471c-4a54-8cb4-29cd95199014,1.230807e-05
4,73b960f9-27b8-4431-9d23-a760e9bbc360,7.163723e-03
...,...,...
4965,a25ee55f-02a3-4f8e-8194-a5f427e14e7c,5.324087e-12
4966,d0d9f45e-7b01-44b3-8d40-514ec338611d,2.031356e-06
4967,f8df22c4-1602-4fab-896d-8820951aae2f,1.069066e-07
4968,b791c69a-f769-4163-afda-051a6fd20a9d,2.733011e-07


In [31]:
# Save the submission dataframe to the `notebook` folder as a csv file
submission.to_csv('../notebooks/submission.csv', index=False)